In [20]:
import matplotlib as mpl
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sklearn.linear_model as linear_model
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from datetime import datetime
import plotly.express as px
import holidays
import sympy
from scipy import optimize
import plotly.io as pio
import plotly.graph_objs as go
pio.renderers.default = "plotly_mimetype+notebook"
height , width= 500,800
import os


# Put-Call Parity

**Forward Contract:** is a contract with a delivery price $K$ that obligates its holder to buy one share of the stock at expiration time $T$ in exchange for payment $K$. At expiration the value of the forward contract is $S(T) - K$. Let $f(t,x)$ denote the value of the forward contract at earlier times $t\in [0,T]$ if the stock price at time $\tau$ is $S(t)=x$, the value of the forward contract at time $\tau$ is :

\begin{align}

f(\tau,x) = x - e^{-r(T-\tau)}K

\end{align}

- Using no arbitrage pricing the payoff of the forward contract agrees with the payoff of a portfolio that is *long one call* and *short one put*, which can be summarized using this mathematical notation:

\begin{align}

f(t,x) &= c(\tau,x) - p(\tau,x)\\

x - e^{-r(T-\tau)}K &= c(\tau,x) - p(\tau,x)

\end{align}

# Parity Relationships 



- <span style = "color:red">short put</span> +<span style = "color:red">short stock</span> = <span style = "color:red">short call</span>

\begin{align}

-P_{0} - S_0 = -C_{0}

\end{align}

- <span style = "color:green">long put</span> + <span style = "color:green">long stock</span> = <span style = "color:green">long  call</span>

\begin{align}

P_{0} + S_0 = C_0

\end{align}

- <span style = "color:green">long  call</span> - <span style = "color:red">short put</span> = <span style = "color:green">long stock</span>

\begin{align}

C_0 - P_{0} = S_{0}

\end{align}

- <span style = "color:red"> short call </span> + <span style = "color:green">long put</span> = <span style = "color:red">short stock</span>

\begin{align}

-C_0 + P_{0} = -S_{0}

\end{align}

- <span style = "color:green">long  call</span> + <span style = "color:red">short stock</span> = <span style = "color:green" >long put</span>

\begin{align}

C_0 - S_0 = P_0

\end{align}

- <span style = "color:red"> short call </span> + <span style = "color:green">long stock</span> = <span style = "color:red">short put</span>

\begin{align}

-C_0 + S_0 = -P_{0}

\end{align}


# Option Strategies | Vertical Spread

## Long Call Vertical Spread
- Aka bull call spread

1. Long lower strike
2. Short higher strike
3. Establish a debit
4. Buy/Sell same number of calls with same expiration but different strike prices
5. Max value is the differential between the two strike prices
6. Min value occurs at the lower strike price
7. Breakeven occurs at the lower strike price plus debit
8. Max profit occurs at the max value

## Short Call Vertical Spread

1. Short lower strike
2. Long higher strike
3. Establish a credit
4. Buy/Sell same number of calls with same expiration but different strike prices
5. Max value is the differential between the two strike prices
6. Min value occurs at the lower strike price
7. Breakeven occurs at the lower strike price plus credit accrued
8. Max profit occurs at the minimum value

In [8]:
spot_s = np.linspace(165,180,num = 31)
c170 = pd.Series(np.maximum(spot_s-170,0) - 4.55,index = spot_s,name = "+C(170)")
c1725 = -1*pd.Series(np.maximum(spot_s - 172.5,0)-3.35,index = spot_s, name = "-C(172.5)")

P = pd.concat([c170,c1725],axis = 1)
P['Payoff'] = P.sum(axis = 1)
bullspread  = px.line(P,title = "C(170)-C(172.5) Call Spread",width=width,height=height)
bullspread.add_vline(x = 172.5,annotation=dict(text = "Max Profit Reached at St >$172.5"))
bullspread.add_hline(y = 0,annotation=dict(text ="$0 PNL"))
bullspread.add_vline(x = 171.2, annotation=dict(text = "$171.2"))

In [10]:
#pval = pd.Series(-np.maximum(spot_s-172.5,0) + np.maximum(spot_s-175,0) + 0.85,index = spot_s,name = "-C(172.5) + C(175)")
c175 = pd.Series(np.maximum(spot_s-175,0)-2.5,index= spot_s, name = "C(175)")
c17250 = -1*pd.Series(np.maximum(spot_s-172.5,0)-3.35,index = spot_s, name ="C(172.5)")
pval = pd.concat([c17250,c175],axis = 1)
pval['Short_BC_Spread'] = pval.sum(axis = 1)
short_bspread = px.line(pval, title = "-C(172.5) + C(175) Short Call Spread",width=width,height=height)
short_bspread.add_hline(y = 0,annotation=dict(text ="$0 PNL"))
short_bspread.add_vline(x = 173.35,annotation=dict(text ="BE @ $173.35"))

## Short Vertical Put Spread

1. Short higher strike put
2. Long lower strike put
3. Establish a credit
4. Buy/Sell same number of contracts with same expiration but different strike prices
5. Max value is the differential between the strike prices
6. Max value begins when stock price reaches lower strike at expiration
7. Minimum value begins when stock price reaches the higher strike price. 
8. Max profit occurs at the minimum value
9. Breakeven occurs at the higher price less the credit accrued. 

## Long Vertical Put Spread

1. Long higher strike put
2. Short lower strike put
3. Establish a debit
4. Buy/Sell same number of contracts with same expiration but different strike prices
5. Max value is the differential between the strike prices. 
6. Max value occurs when stock prices reaches lower strike at expiration
7. Minimum value begins when stock prices reaches higher strike price
8. Max profit occurs at the max value
9. Break even occurs at the higher strike less debit

In [15]:
states_space = np.linspace(128,138,num = 21)

portf = pd.DataFrame(pd.Series(np.maximum(135-states_space,0)-np.maximum(132-states_space,0)-0.77,index= states_space,name = "P(135)-P(132)"))
putspread_fig = px.line(portf, title = "P(135)-P(132) Long Vertical Put Spread",height=height,width=width)
putspread_fig.add_hline(y = 0,annotation=dict(text ="$0 PNL"))
putspread_fig.add_vline(x = 135-.77,annotation=dict(text ="BE @ $134.23"))

In [18]:
bull_put = pd.DataFrame(pd.Series(np.maximum(129-states_space,0)-np.maximum(132-states_space,0)+0.43,index = states_space,name = "Bull_Put_Spread"))
bull_putfig = px.line(bull_put,title = "P(29)-P(32) Short Vertical Put",width=width,height=height)
bull_putfig.add_hline(y = 0,annotation=dict(text ="$0 PNL"))
bull_putfig.add_vline(x = 131.57,annotation=dict(text ="BE @ $131.57"))

## Long Straddle


1. Buy same number of calls and puts with the same strike price and expiration
2. Establish a debit
3. DBEP is strike less debit
4. UBEP is strike plus debit

- benefits from increasing volatility

## Short Straddle

1. Sell same number of calls and puts with same strike and expiration
2. Establish a credit
3. DBEP is strike less credit
4. UBEP is strike plus credit

## Long Strangle

1. Buy OTM Call
2. Buy OTM Put
3. DBEP is put strike less debit
4. UBEP is call strike plus debit

## Iron Butterfly

- A short straddle combined with a long strangle

- The long strangle is used to cap your losses in case the stock price moves a lot in either direction. 

In [19]:

states = np.linspace(170,210,num = 81)
liron_butterfly = pd.DataFrame(-np.maximum(190-states,0)-np.maximum(states-190,0)+np.maximum(states-200,0)+np.maximum(180-states,0)+6.74,index = states, columns=['Long_Iron_Butterfly'])
liron_butterflyfig = px.line(liron_butterfly,title = 'Long Iron Butterfly',width=width,height=height)
liron_butterflyfig.add_hline(y = 0, annotation=dict(text="$0 PNL"))